### 令和6年6月30日(日)
PCA分析の結果を特徴量に加える

#### 1. 読み込み

In [1]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors , KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
# LGBMClassifier
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from hyperopt import hp, fmin, tpe, Trials
import warnings
import sys
sys.path.append('../')
from my_utils import MyUtils
warnings.filterwarnings('ignore')

INPUT_DIR = "../input/"
RANDOM_STATE = 10
OUTPUT_PATH = "C:/Users/gwsgs/workSpace/GCIcomp2/02.（公開）コンペ2/output/"
remove_outliers_columns = [ "AMT_INCOME_TOTAL", "AMT_ANNUITY", "DAYS_ID_PUBLISH",]
exel_path = 'score_record.xlsx' 
utils = MyUtils(input_path=INPUT_DIR, output_path=OUTPUT_PATH, exel_path=exel_path)  
params = utils.params

In [2]:
# データの読み込み
train, test, X_train, y_train, sample_sub = utils.load_data()
HAS_NAN_INIT = utils.missing_columns() # 欠損値を含む特徴量
NUMERICAL_VARIABLES = train.select_dtypes(include=[np.number]).columns.tolist() # 数値変数を含む特徴量
# 量的データの特徴量
BINARY_FEATURES = [col for col in NUMERICAL_VARIABLES if train[col].nunique() == 2] # 2値変数
NUMERICAL_FEATURES = ["CNT_CHILDREN","AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY","AMT_GOODS_PRICE",
                        "REGION_POPULATION_RELATIVE","DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION",
                        "DAYS_ID_PUBLISH","OWN_CAR_AGE","EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3",
                        "OBS_30_CNT_SOCIAL_CIRCLE","DEF_30_CNT_SOCIAL_CIRCLE","OBS_60_CNT_SOCIAL_CIRCLE",
                        "DEF_60_CNT_SOCIAL_CIRCLE","DAYS_LAST_PHONE_CHANGE","AMT_REQ_CREDIT_BUREAU_HOUR",
                        "AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_YEAR"]# 量的データ
NUMERICAL_HAS_NAN = [col for col in NUMERICAL_FEATURES if col in HAS_NAN_INIT] # 欠損値を含みかつ数値データである特徴量
CATEGORICAL_FEATURES = train.select_dtypes(exclude=[np.number]).columns.tolist() # カテゴリカル変数を含む特徴量
CATEGORICAL_HAS_NAN = [col for col in CATEGORICAL_FEATURES if col in HAS_NAN_INIT] # 欠損値を含みかつカテゴリカルデータである特徴量

#### 2. 前処理と特徴量作成

In [3]:
# 外れ値の除去
train = utils.remove_outliers(train, remove_outliers_columns, quantile=0.99999)

# # 標準化
# scaler = StandardScaler()
# train[NUMERICAL_FEATURES] = scaler.fit_transform(train[NUMERICAL_FEATURES])

# 特徴量の作成
# CODE_GENDERが1かつNAME_FAMILY_STATUSがmarriedの場合 = 1,  新しく特徴量[female_marride]を作成
# OCCUPATION_TYPEのLow-skill Laborersを1、それ以外を0に変換。新しく特徴量[is_low_skill]を作成
train["is_low_skill"] = train["OCCUPATION_TYPE"].apply(lambda x: 1 if x == "Low-skill Laborers" else 0)
test["is_low_skill"] = test["OCCUPATION_TYPE"].apply(lambda x: 1 if x == "Low-skill Laborers" else 0)

# カテゴリカルデータの欠損値をUNKNOWNで補完
for column in CATEGORICAL_HAS_NAN:
    train[column] = train[column].fillna("UNKNOWN")
    test[column] = test[column].fillna("UNKNOWN")

# CODE_GENDERのXNAをUNKNOWNに変換
train["CODE_GENDER"] = train["CODE_GENDER"].replace("XNA", "F")
test["CODE_GENDER"] = test["CODE_GENDER"].replace("XNA", "F")

# カテゴリカルデータをターゲットエンコーディング Holdout Target Encoding https://www.codexa.net/target_encoding/
train, test = utils.target_label_count_encoding(train, test, CATEGORICAL_FEATURES)

AMT_INCOME_TOTAL_(171202,)
外れ値：113305    13500000.0
159447     9000000.0
Name: AMT_INCOME_TOTAL, dtype: float64
AMT_ANNUITY_(171202,)
外れ値：33614     230161.5
128408    258025.5
Name: AMT_ANNUITY, dtype: float64
DAYS_ID_PUBLISH_(171202,)
外れ値：26886   -6383
30640   -7197
Name: DAYS_ID_PUBLISH, dtype: int64


In [4]:
# 数値型の欠損値の補完を行う
train["CNT_FAM_MEMBERS"].fillna(train["CNT_FAM_MEMBERS"].median(), inplace=True)
test["CNT_FAM_MEMBERS"].fillna(train["CNT_FAM_MEMBERS"].median(), inplace=True)

for i in utils.missing_columns():
    train[i].fillna(train[i].median(), inplace=True)
    test[i].fillna(train[i].median(), inplace=True)

In [5]:
# 特徴量の作成
# DAYS_LAST_PHONE_CHANGEを年に変換
train["YEARS_PHONR_CHANGE"] = train["DAYS_LAST_PHONE_CHANGE"] / -365
test["YEARS_PHONR_CHANGE"] = test["DAYS_LAST_PHONE_CHANGE"] / -365
# DAYS_EMPLOYEDを年に変換
train["YEARS_EMPLOYED"] = train["DAYS_EMPLOYED"] / 365
test["YEARS_EMPLOYED"] = test["DAYS_EMPLOYED"] / 365
# DAYS_REGISTRATIONを年に変換
train["YEARS_REGISTRATION"] = train["DAYS_REGISTRATION"] / -365
test["YEARS_REGISTRATION"] = test["DAYS_REGISTRATION"] / -365
# DAYS_ID_PUBLISHを年に変換
train["YEARS_ID_PUBLISH"] = train["DAYS_ID_PUBLISH"] / -365
test["YEARS_ID_PUBLISH"] = test["DAYS_ID_PUBLISH"] / -365
#  1 if AMT_GOODS_PRICEnan==nan other = 0
train["AMT_GOODS_PRICE_ISNAN"] = train["AMT_GOODS_PRICE"].isnull().astype(int)
test["AMT_GOODS_PRICE_ISNAN"] = test["AMT_GOODS_PRICE"].isnull().astype(int)
# own car and realty = 4 , own realty = 2 , own car = 1 , other = 0 commentと異なる
train["OWN_CAR_REALTY"] = train["FLAG_OWN_CAR"] + train["FLAG_OWN_REALTY"]
test["OWN_CAR_REALTY"] = test["FLAG_OWN_CAR"] + test["FLAG_OWN_REALTY"]
# DAYS_EMPLOYEDが0以上の場合、1 それ以外は0
train["not_employed"] = train["DAYS_EMPLOYED"].apply(lambda x: 1 if x > 0 else 0)
test["not_employed"] = test["DAYS_EMPLOYED"].apply(lambda x: 1 if x > 0 else 0)

# 参考 https://www.kaggle.com/competitions/home-credit-default-risk/discussion/64821
# AMT_CREDITとAMT_ANNUITYの比率
train["CREDIT_TO_ANNUITY_RATIO"] = train["AMT_CREDIT"] / train["AMT_ANNUITY"]
test["CREDIT_TO_ANNUITY_RATIO"] = test["AMT_CREDIT"] / test["AMT_ANNUITY"]
# AMT_CREDITとAMT_GOODS_PRICEの比率
train["CREDIT_TO_GOODS_RATIO"] = train["AMT_CREDIT"] / train["AMT_GOODS_PRICE"]
test["CREDIT_TO_GOODS_RATIO"] = test["AMT_CREDIT"] / test["AMT_GOODS_PRICE"]
# AMT_CREDITとAMT_ANNUITYの比率
train["AMT_CREDIT_TO_AMT_ANNUITY"] = train["AMT_CREDIT"] / train["AMT_ANNUITY"]
test["AMT_CREDIT_TO_AMT_ANNUITY"] = test["AMT_CREDIT"] / test["AMT_ANNUITY"]
# DAYS_BIRTH/ -365
train["YEARS_BIRTH"] = train["DAYS_BIRTH"] / -365
test["YEARS_BIRTH"] = test["DAYS_BIRTH"] / -365

In [6]:
days_feature = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH"]
# # 年単位でビニング
# train = utils.from_days_to_year_bin(train, days_feature)
# test = utils.from_days_to_year_bin(test, days_feature)
# 月単位でビニング
train = utils.from_days_to_month_bin(train, days_feature)
test = utils.from_days_to_month_bin(test, days_feature)
# 週単位でビニング
train = utils.from_days_to_week_bin(train, days_feature)
test = utils.from_days_to_week_bin(test, days_feature)

In [7]:
# 線形回帰によって新たな特徴量を作成
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
X_test = test.values
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
model = LinearRegression()
model.fit(X, y)
train['lr'] = model.predict(X_train)
test['lr'] = model.predict(X_test)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# PCAによる次元削減結果を特徴量として追加
pca = PCA(n_components=2)
pca.fit(X)
X_pc = pca.transform(X)
train['pca1'] = X_pc[:, 0]
train['pca2'] = X_pc[:, 1]
pca.fit(X_test)
y_pc = pca.transform(X_test)
test['pca1'] = y_pc[:, 0]
test['pca2'] = y_pc[:, 1]

In [10]:
X_train, X_valid, y_train, y_valid, X_test, X, y = utils.split_data(train, test)

### 3. 観察

In [ ]:
print(f"X_train:{X_train.shape}")
print(f"X_valid:{X_valid.shape}")
print(f"X_test:{X_test.shape}")
print(f"X:{X.shape}")
print(f"y_train:{y_train.shape}")
print(f"y_valid:{y_valid.shape}")
print(f"y:{y.shape}")
print(f"train:{train.shape}")
print(f"test:{test.shape}")

In [ ]:
# import sweetviz as sv

# # Sweetvizの分析レポートを作成（学習データとテストデータの比較、目的変数を指定）
# report = sv.compare([train, "Train Data"], [test, "Test Data"], target_feat='TARGET')

# # レポートをHTMLファイルとして保存
# report.show_html(open_browser=True)

In [ ]:
sns.scatterplot(data=train, x='EXT_SOURCE_1',y='DAYS_BIRTH_bin_week', size=0.1)

## 4. 予測

In [ ]:
# 複数のモデルを使って予測値を出力
utils.multi_model_predict(X_train, y_train, X_valid, y_valid, all=False)

In [11]:
def objective(args):
    lgb = LGBMClassifier(
        num_leaves = args['num_leaves'],
        max_depth = args['max_depth'],
        n_estimators = args['n_estimators'],
        learning_rate = args['learning_rate'],
        # min_child_samples = args['min_child_samples'],
        reg_lambda = args['reg_lamb'],
        bagging_freq = args['bagging_freq'],
        bagging_fraction = args['bagging_fraction'],
        feature_fraction = args['feature_fraction'],
        min_data_in_leaf = args['min_data_in_leaf'],
        random_state = params['random_state'],
        objective = params['objective'],
        metric = params['metric'],
        verbosity = params['verbosity'],
        boosting_type = params['boosting_type'],
        early_stopping_round = 50
        )
    lgb.fit(X_train, y_train,
            eval_set = [(X_valid, y_valid)]
        )
    lgb_valid_pred = lgb.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, lgb_valid_pred)
    return -1.0 * auc   

In [12]:
import datetime
n_iter = 100
trials = Trials()
time = datetime.datetime.now().strftime('%Y%m%d%H%M')
best = fmin(
    objective,
    space = params,
    algo=tpe.suggest,
    max_evals=n_iter,
    trials=trials,
    verbose=-1, # 試行の過程を出力
    early_stop_fn=None,
    show_progressbar = True,
    # trials_save_file=f"{output_path}_trials_save_file_{time}.pkl"
)

100%|██████████| 100/100 [05:59<00:00,  3.59s/trial, best loss: -0.7590601010792202]


In [ ]:
# train全体に対して学習
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
lgb = LGBMClassifier(
        learning_rate = best['learning_rate'],
        objective = params['objective'],
        max_depth = best['max_depth'],
        # min_child_samples = best['min_child_samples'],
        n_estimators = best['n_estimators'],
        num_leaves = best['num_leaves'],
        reg_lambda= best['reg_lamb'],
        bagging_freq = best['bagging_freq'],
        bagging_fraction = best['bagging_fraction'],
        feature_fraction = best['feature_fraction'],
        min_data_in_leaf = best['min_data_in_leaf'],
        verbose = -1,
        random_state = RANDOM_STATE,)
lgb.fit(X, y)

print(best)

In [ ]:
utils.save_score_to_exel(utils.exel_path, best, trials.best_trial['result']['loss'] * -1, time)

In [ ]:
dic = {}
for i,  column in enumerate(test.columns):
    dic[i] = column
dic

In [ ]:
lgbm.plot_importance(lgb,figsize=(5, 15))

#### 4. 予測結果の作成
最後にテストデータに対して予測を行い、提出用のcsvファイルを作成します。

In [ ]:
# テストデータに対する予測値の作成
# pred = lgb.predict_proba(X_test, num_iteration=lgb.best_iteration)[:, 1]
pred = lgb.predict_proba(X_test)[:, 1]

In [ ]:
# 予測結果を提出用のフォーマットに格納
sample_sub['TARGET'] = pred

In [ ]:
# 提出用のcsvファイルを作成
# formatを指定して現在時刻を取得（yyyyMMddhhmm）
sample_sub.to_csv(f'{OUTPUT_PATH}/{time}.csv',index=False)

以上で、Home Credit Default Riskコンペのチュートリアルは終了です。今回は、50種類ある特徴量のうち5種類しか使用していないので、まだまだ改善の余地があります。この後は、このnotebookやこれまでの教材を参考にして、さらなるスコアの向上を目指してください！